# Data Acquisition and Ingestion

In [1]:
# Import libraries
import os, pathlib, pandas as pd, requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup

# Set up data directory
DATA_RAW = pathlib.Path('data/raw')
DATA_RAW.mkdir(parents=True, exist_ok=True)

# Load API key from .env
load_dotenv()
ALPHA_KEY = os.getenv('ALPHAVANTAGE_API_KEY')

In [2]:
# Helper: validate DataFrame
def validate_df(df, required_cols, dtypes_map):
    msgs = {}
    missing = [c for c in required_cols if c not in df.columns]
    if missing:
        msgs['missing_cols'] = missing
    for col, dtype in dtypes_map.items():
        if col in df.columns:
            try:
                if dtype == 'datetime64[ns]':
                    pd.to_datetime(df[col])
                elif dtype == 'float':
                    pd.to_numeric(df[col])
            except Exception as e:
                msgs[f'dtype_{col}'] = str(e)
    msgs['na_total'] = int(df.isna().sum().sum())
    msgs['shape'] = df.shape
    return msgs

In [3]:
## Download stock data from API or yfinance
SYMBOL = 'NVDA'
if ALPHA_KEY:
    url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY_ADJUSTED',
        'symbol': SYMBOL,
        'outputsize': 'compact',
        'apikey': ALPHA_KEY,
        'datatype': 'json'
    }
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js.keys() if 'Time Series' in k][0]
    series = js[key]
    df_api = pd.DataFrame(series).T.rename_axis('date').reset_index()
    df_api = df_api[['date', '5. adjusted close']].rename(columns={'5. adjusted close': 'adj_close'})
    df_api['date'] = pd.to_datetime(df_api['date'])
    df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='6mo', interval='1d',auto_adjust=True).reset_index()
    # Use 'Adj Close' if present, else use 'Close'
    if 'Adj Close' in df_api.columns:
        df_api = df_api[['Date', 'Adj Close']]
        df_api.columns = ['date', 'adj_close']
    elif 'Close' in df_api.columns:
        df_api = df_api[['Date', 'Close']]
        df_api.columns = ['date', 'adj_close']
    else:
        raise ValueError("No 'Adj Close' or 'Close' column found in yfinance data")

[*********************100%***********************]  1 of 1 completed


In [25]:
# Validate and save API data
msgs = validate_df(df_api, ['date','adj_close'], {'date':'datetime64[ns]','adj_close':'float'})
print(msgs)
df_api.to_csv(DATA_RAW / 'api_aapl.csv', index=False)

{'shape': (125, 2), 'na_total': np.int64(0), 'dtype_mismatch_adj_close': 'Expected float, got float64'}


In [29]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from pathlib import Path
import os
from datetime import datetime
import re
import json

# Define missing functions
def validate_df(df, required_cols=None, dtypes_map=None):
    """Validate dataframe and return validation messages"""
    msgs = {}
    
    # Check shape
    msgs['shape'] = df.shape
    
    # Check for NA values
    msgs['na_total'] = df.isna().sum().sum()
    
    # Check required columns
    if required_cols:
        missing_cols = set(required_cols) - set(df.columns)
        if missing_cols:
            msgs['missing_cols'] = list(missing_cols)
    
    # Check data types
    if dtypes_map:
        for col, expected_dtype in dtypes_map.items():
            if col in df.columns:
                actual_dtype = str(df[col].dtype)
                if actual_dtype != expected_dtype:
                    msgs[f'dtype_mismatch_{col}'] = f"Expected {expected_dtype}, got {actual_dtype}"
    
    return msgs

def safe_filename(prefix="data", meta=None):
    """Generate a safe filename with timestamp and metadata"""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    if meta:
        meta_str = "_".join([f"{k}_{v}" for k, v in meta.items()])
        filename = f"{prefix}_{meta_str}_{timestamp}.csv"
    else:
        filename = f"{prefix}_{timestamp}.csv"
    
    # Remove any unsafe characters
    filename = re.sub(r'[<>:"/\\|?*]', '_', filename)
    return filename

# Define data directory
DATA_RAW = Path("./data")
DATA_RAW.mkdir(exist_ok=True)

# Your scraping code - Modified to handle Alpha Vantage JSON API
SCRAPE_URL = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol=nvda&apikey=Q5H86S3MHIMLXE51" 
headers = {"User-Agent": "AFE-Course-Notebook/1.0 (contact: instructor@example.edu)"}

try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30)
    resp.raise_for_status()
    
    # Parse JSON response from Alpha Vantage API
    data = resp.json()
    
    # Check if we have monthly time series data
    if 'Monthly Time Series' in data:
        monthly_data = data['Monthly Time Series']
        df_scrape = pd.DataFrame.from_dict(monthly_data, orient='index')
        
        # Rename columns to be more descriptive
        df_scrape.columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        
        # Convert data types to numeric
        for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
            df_scrape[col] = pd.to_numeric(df_scrape[col], errors='coerce')
        
        print("Successfully scraped NVDA monthly data from Alpha Vantage API")
        
    elif 'Error Message' in data:
        raise ValueError(f"API Error: {data['Error Message']}")
    else:
        raise ValueError("No monthly time series data found in API response")
        
except Exception as e:
    print("API call failed (demoing with inline HTML).", e)
    html = """
    <table>
      <tr><th>Ticker</th><th>Price</th></tr>
      <tr><td>AAA</td><td>101.2</td></tr>
      <tr><td>BBB</td><td>98.7</td></tr>
    </table>
    """
    soup = BeautifulSoup(html, 'html.parser')
    rows = []
    for tr in soup.find_all('tr'):
        cells = [td.get_text(strip=True) for td in tr.find_all(['td','th'])]
        if cells:
            rows.append(cells)
    header, *data = rows
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')

msgs2 = validate_df(df_scrape, required_cols=list(df_scrape.columns), dtypes_map={})
print(msgs2)

fname2 = safe_filename(prefix="scrape", meta={"site": "example", "table": "markets"})
out_path2 = DATA_RAW / fname2
df_scrape.to_csv(out_path2, index=False)
print("Saved:", out_path2)

Successfully scraped NVDA monthly data from Alpha Vantage API
{'shape': (309, 5), 'na_total': np.int64(0)}
Saved: data\scrape_site_example_table_markets_20250818_044718.csv
